# Create Agents to Research and Write an Article

In this notebook, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.30.11 crewai_tools==0.2.6 langchain_community==0.0.38 langchain==0.1.20
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-4o`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os

#from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

# Testing local llms
llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434/v1")

### Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool, CodeDocsSearchTool
codeDocTool = CodeDocsSearchTool()
youtubeChannelTool= YoutubeChannelSearchTool()
youtubeVideoTool = YoutubeVideoSearchTool()
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [15]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    tools=[scrape_tool, search_tool],
   #llm=llm
)

##############################################################
# Agent: Writer
#
##############################################################
writer = Agent(
    role="Content Writer",
    goal="Write insightful and easy to follow "
         "tutorial about the topic: {topic}",
    backstory="You're working on a writing "
              "a new tutorial about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You state your sources.",
    allow_delegation=False,
    verbose=True,
    #llm=llm
)

##############################################################
# Agent: Editor
#
##############################################################
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. "
         "Ensure that the blog post is written for beginners",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    #llm=llm
)
##############################################################
# Agent: Researcher
#
##############################################################
researcher = Agent(
    role="Tech Researcher",
    goal="To find excellent sources of information for the writer to use as a starting point.",
    tools=[search_tool, scrape_tool,codeDocTool, youtubeChannelTool, youtubeVideoTool],
    verbose=True,
    backstory=(
        "As a Tech Researcher, your prowess in "
        "navigating and extracting critical "
        "information from documentation is unmatched."
        "You are an expert in reading code and translating it to plain english"    
    ),
    #llm=llm,
    
   # allow_delegation=False
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the ease of use for this tutorial, "
            "and latest use cases on {topic}.\n"
        "2. Plan the article for those with limited knowledge of python.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    tools=[search_tool, scrape_tool]
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft an easy to follow "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    tools=[search_tool, scrape_tool],
    context = [plan]
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    tools=[search_tool, scrape_tool]
)

### All Task

In [16]:
################################################################################
# Task: Plan
#
################################################################################
plan = Task(
    description=(
        "1. Choose a task that is easy to replicate for the following topic:  "
            "{topic}.\n"
        "2. Plan the article for those with limited knowledge of python.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "It should be formatted in a sequential step by step manner.",
    agent=planner,
    tools=[search_tool, scrape_tool]
)

################################################################################
# Task: Research
#
################################################################################
research = Task(
    description=("Given the current plan by the Content Planner make sure to do the following.\n"
        "1. Determine which sites are the best to search. For the planned topic of {topic}.\n"
        "2. Make a summary of sources the Writer can use as a reference when making the article."
    ),
    expected_output="A structured list of resources to use a brief description for each source.",
    agent=researcher,
    tools=[search_tool, scrape_tool,codeDocTool, youtubeChannelTool, youtubeVideoTool],
    context=[plan]
)

################################################################################
# Task: Write
#
################################################################################
write = Task(
    description=(
        "1. Using the outline provided by the content planner and tech researcher make a"
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. This should be written in a follow along style that the reader can replicate.\n"
        "6. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    tools=[search_tool, scrape_tool],
    context = [plan,research]
)

################################################################################
# Task: Edit
#
################################################################################
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "ensure that there is plenty of content."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    context=[plan,research,write],
    output_file="final_article.md"
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, researcher, writer, editor],
    tasks=[plan, research, write, edit],
    process=Process.sequential,
    verbose=True,
    
)
#Process.hierarchical
#Process.sequential
#manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
 #                          temperature=0.7)

2024-06-15 10:16:50,915 - 14576 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Building a Simple blog writing Crew with CrewAI"})

- Display the results of your execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown
Markdown(result)

my best complete final answer to the task.

```markdown
## Building a Simple Blog Writing Crew with CrewAI

### Introduction
In the ever-evolving world of content creation, staying ahead of the curve is crucial. Enter **CrewAI**, a cutting-edge framework designed to orchestrate role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks. This article aims to guide you through building a simple blog writing crew using CrewAI, showcasing its potential to streamline your blog writing process and enhance productivity.

### Understanding CrewAI Components
Before diving into the setup, it’s essential to understand the key components of CrewAI:

**Agents**: Agents in CrewAI are autonomous entities with defined roles and goals. For instance, a 'Researcher' might be tasked with gathering information, while a 'Writer' focuses on crafting engaging content.

**Tasks**: These are individual assignments allocated to agents. Each task comes with a description and an expected output, ensuring clarity and direction.

**Tools**: Agents utilize various tools to accomplish their tasks. These could include web scraping tools, API integration functionalities, or language models.

**Processes**: CrewAI orchestrates task execution through processes. These processes can be sequential or hierarchical, guiding agents through their tasks efficiently.

### Initial Setup
Setting up CrewAI involves a few straightforward steps:

1. **Installation**:
   To install CrewAI, use the following command:
   ```bash
   pip install crewai
   ```

   For additional tools, use:
   ```bash
   pip install 'crewai[tools]'
   ```

2. **Setting Up Environment Variables**:
   Ensure you have the necessary API keys by setting up environment variables:
   ```python
   import os
   os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
   os.environ["SERPER_API_KEY"] = "Your Key"
   ```

### Creating Your First Crew
Now, let’s create a blog writing crew with CrewAI. Follow these steps:

1. **Define Agents and Their Roles**:
   ```python
   from crewai import Agent, Task, Crew, Process
   from crewai_tools import SerperDevTool

   search_tool = SerperDevTool()

   researcher = Agent(
       role='Senior Research Analyst',
       goal='Uncover cutting-edge developments in AI and data science',
       backstory="""You work at a leading tech think tank.
       Your expertise lies in identifying emerging trends.
       You have a knack for dissecting complex data and presenting actionable insights.""",
       verbose=True,
       tools=[search_tool]
   )

   writer = Agent(
       role='Tech Content Strategist',
       goal='Craft compelling content on tech advancements',
       backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
       You transform complex concepts into compelling narratives.""",
       verbose=True
   )
   ```

2. **Create Tasks for Agents**:
   ```python
   task1 = Task(
       description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
       Identify key trends, breakthrough technologies, and potential industry impacts.""",
       expected_output="Full analysis report in bullet points",
       agent=researcher
   )

   task2 = Task(
       description="""Using the insights provided, develop an engaging blog
       post that highlights the most significant AI advancements.
       Your post should be informative yet accessible, catering to a tech-savvy audience.
       Make it sound cool, avoid complex words so it doesn't sound like AI.""",
       expected_output="Full blog post of at least 4 paragraphs",
       agent=writer
   )
   ```

3. **Initialize and Configure the Crew**:
   ```python
   crew = Crew(
       agents=[researcher, writer],
       tasks=[task1, task2],
       verbose=2
   )
   ```

4. **Execute the Crew**:
   ```python
   result = crew.kickoff()
   print("######################")
   print(result)
   ```

### Example Project: Blog Writer Assistant
To illustrate, let’s walk through a practical example of a blog writing crew with a 'Senior Research Analyst' and a 'Tech Content Strategist'. The researcher agent gathers the latest AI advancements, while the writer agent crafts a compelling blog post based on the research.

By defining clear roles, specific tasks, and utilizing the appropriate tools, you can seamlessly automate the content creation process. This not only saves time but also ensures high-quality and consistent output.

### Advanced Tips and Best Practices
To optimize your CrewAI setup, consider these advanced tips:

1. **Securely Manage API Keys**:
   Always store API keys securely using environment variables or dedicated vault services to prevent unauthorized access.

2. **Optimize Agent Performance**:
   Regularly update and fine-tune your agents and tools to ensure optimal performance. Utilize logging and monitoring to identify and address any potential bottlenecks.

3. **Integrate Additional Tools and Services**:
   Enhance the functionality of your agents by integrating additional tools and services. This could include advanced language models, data analytics tools, or custom APIs.

### Call to Action
Ready to revolutionize your blog writing process? Try building your blog writing crew using CrewAI today. For further assistance, explore the [CrewAI GitHub Repository](https://github.com/joaomdmoura/crewAI), [CrewAI Documentation](https://docs.crewai.com/), and join the community forums for additional support and resources.

### Resources
- [CrewAI GitHub Repository](https://github.com/joaomdmoura/crewAI)
- [CrewAI Documentation](https://docs.crewai.com/)
- [Composio Blog on CrewAI Examples](https://blog.composio.dev/crewai-examples/)

By following this guide, you will be well on your way to creating a powerful and efficient blog writing crew with CrewAI. Happy blogging!
```

In [16]:
from IPython.display import Markdown, display
display(Markdown("./final_article.md"))

my best complete final answer to the task.


# Building a Simple Blog Writing Crew with CrewAI

## Introduction

Welcome to the future of blog writing! In this article, we’ll explore how you can build a simple yet effective blog writing crew using CrewAI, an open-source framework for orchestrating collaborative AI agents. Whether you're a seasoned blogger or just starting out, CrewAI can help streamline your content creation process, enabling you to produce high-quality articles with minimal effort.

## Understanding AI Agents

Before diving into CrewAI, it's essential to understand the concept of AI agents. AI agents are software entities that perform tasks autonomously. They can perform various functions such as translation, summarizing, reasoning, and task automation. The power of AI agents lies in their ability to handle complex tasks through collaborative intelligence, making them ideal for content creation.

## Introduction to CrewAI

### What is CrewAI?

CrewAI is an open-source framework designed to orchestrate role-playing and autonomous AI agents. It empowers agents to work together seamlessly, tackling complex tasks efficiently. Whether you're building a smart assistant platform or an automated content creation team, CrewAI provides the backbone for sophisticated multi-agent interactions.

### Key Features of CrewAI

- **Role-based Agents:** Define agents with specific roles, goals, and backstories to simulate human-like collaboration.
- **Task Management:** Define tasks with tools and dynamically assign them to agents.
- **Inter-agent Delegation:** Agents can delegate tasks to other agents, enhancing problem-solving efficiency.

## Setting Up CrewAI

### Prerequisites

Before you start, ensure you have a basic understanding of Python. You'll also need to install CrewAI and necessary libraries. Follow these steps to set up your environment:

1. **Install CrewAI and Langchain:**
    ```bash
    pip install crewai langchain-google-genai
    ```

2. **Set Up Your Virtual Environment:**
    ```bash
    python -m venv crewai_env
    source crewai_env/bin/activate  # On Windows use `crewai_env\Scripts\activate`
    ```

## Building a Collaborative AI Crew for Blog Writing

### Defining Roles

To create an effective blog writing crew, you need to define specific roles for each agent. Here are the roles we'll be using:

- **Idea Analyst:** Analyzes and expands on blog ideas.
- **Writer:** Writes compelling articles.
- **Editor:** Edits and formats content.

### Sample Code for Defining Agents

Here's a detailed code example for defining each agent:

```python
from crewai import Agent
from langchain_google_genai import GoogleGenerativeAI

GOOGLE_API_KEY = "Your Key"
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

idea_analyst = Agent(
    role="Idea Analyst",
    goal="Comprehensively analyze an idea to prepare blueprints for the article",
    backstory="Experienced content analyst",
    llm=llm,
    verbose=True
)

writer = Agent(
    role="Writer",
    goal="Write compelling articles",
    backstory="Renowned fiction-writer",
    llm=llm,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit and format content",
    backstory="Experienced editor",
    llm=llm,
    tools=[FileTools.write_file],
    verbose=True
)
```

## Defining Tasks for Each Agent

Next, you'll need to define tasks for each agent. Here's how to do it:

### Task Definitions

```python
from textwrap import dedent

class CreateTasks:
    @staticmethod
    def expand_idea(idea):
        return dedent(f"""Analyze the given task {idea}. Prepare comprehensive pin-points for the article.""")
    
    @staticmethod
    def write():
        return dedent("Write a compelling article based on the blueprint ideas.")
    
    @staticmethod
    def edit():
        return dedent("Edit, format, and save the content.")
```

## Assembling the Crew and Executing Tasks

Finally, you'll assemble the agents and tasks to create and run your blog writing crew.

### Creating and Running the Crew

```python
from crewai import Crew, Task
from tasks import CreateTasks

class ContentWritingCrew:
    def __init__(self, idea):
        self.idea = idea

    def __call__(self):
        tasks = [
            Task(description=CreateTasks.expand_idea(self.idea), agent=idea_analyst),
            Task(description=CreateTasks.write(), agent=writer),
            Task(description=CreateTasks.edit(), agent=editor)
        ]
        crew = Crew(tasks=tasks, agents=[idea_analyst, writer, editor], verbose=True)
        result = crew.kickoff()
        return result

if __name__ == "__main__":
    idea = input("Enter blog idea: ")
    my_crew = ContentWritingCrew(idea=idea)
    result = my_crew()
    print(result)
```

## Real-world Use Cases

While this tutorial focuses on blog writing, AI agents have numerous applications beyond content creation. They can serve as personal AI assistants, code interpreters, virtual instructors, and more. The flexibility and scalability of CrewAI make it suitable for various advanced AI-driven tasks.

## Conclusion

AI agents, particularly when orchestrated using frameworks like CrewAI, represent the future of content creation. By breaking down complex tasks into manageable roles and leveraging the capabilities of large language models, you can streamline your blog writing process and boost productivity. Start experimenting with CrewAI today and take your content creation to the next level!

## Call to Action

Ready to revolutionize your blog writing process? Start using CrewAI today and see how AI agents can enhance your content creation workflow. Don't forget to share your experiences and any questions in the comments below!

## Resources

- [CrewAI GitHub Repository](https://github.com/joaomdmoura/crewAI)
- [Langchain Documentation](https://python.langchain.com/v0.2/docs/introduction/)
- [Google Generative AI Documentation](https://cloud.google.com/generative-ai)
- [Replit Repository](https://replit.com/@Sunil-KumarKu17/CollborativeAIAgent)


This is a well-structured and complete blog post, ready for publication. It provides a thorough introduction to CrewAI, explains the necessary steps to set it up, and offers practical examples for building a collaborative AI crew for blog writing. Additionally, it discusses real-world use cases and encourages readers to start experimenting with CrewAI.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).